<a href="https://colab.research.google.com/github/Sumugaa/Tour-Planner/blob/main/CodeTourPlanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openrouteservice

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import openrouteservice
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

# Initialize the OpenRouteService client
api_key = '5b3ce3597851110001cf6248c787797e0fd749bb99e400679ce6b3c9'
client = openrouteservice.Client(key=api_key)

# Prompt the user for the number of locations they want to provide
num_locations = int(input("Enter the number of locations: "))

# Create an empty list to store the locations
locations = []

# Loop through each location and ask the user to enter its name
for i in range(num_locations):
    name = input(f"Enter the name of location {i+1}: ")
    locations.append(name)

# Create an empty adjacency matrix
adj_matrix = [[0 for i in range(num_locations)] for j in range(num_locations)]

# Loop through each pair of locations and calculate the distance between them
for i in range(num_locations):
    for j in range(num_locations):
        if i != j:
            # Get the coordinates of the two locations
            geolocator = Nominatim(user_agent="tour_planner")
            location_i = geolocator.geocode(locations[i])
            location_j = geolocator.geocode(locations[j])
            coords_i = [location_i.longitude, location_i.latitude]
            coords_j = [location_j.longitude, location_j.latitude]

            # Calculate the distance between the two locations using the OpenRouteService client
            params = {'profile': 'driving-car', 'format': 'geojson'}
            route = client.directions(coordinates=[coords_i, coords_j], **params)
            distance = route['features'][0]['properties']['segments'][0]['distance']

            # Store the distance in the adjacency matrix
            adj_matrix[i][j] = distance

# Print the adjacency matrix
for row in adj_matrix:
    print(row)

import sys
import numpy as np

def tsp_branch_and_bound(adj_matrix):
    num_locations = len(adj_matrix)
    # Create a list to track visited locations
    visited = [False] * num_locations
    # Mark the starting location as visited
    visited[0] = True
    # Initialize variables
    shortest_path = sys.maxsize
    current_path = [0]
    min_path = []
    
    def tsp_util(current_node, visited, current_path, current_cost):
        nonlocal shortest_path, min_path
        if len(current_path) == num_locations:
            # All locations visited, return to starting point
            current_path.append(0)
            current_cost += adj_matrix[current_node][0]
            if current_cost < shortest_path:
                shortest_path = current_cost
                min_path = current_path.copy()
            current_path.pop()
            return
        
        for next_node in range(num_locations):
            if not visited[next_node]:
                visited[next_node] = True
                current_path.append(next_node)
                tsp_util(next_node, visited, current_path, current_cost + adj_matrix[current_node][next_node])
                visited[next_node] = False
                current_path.pop()
    
    tsp_util(0, visited, current_path, 0)
    
    return shortest_path, min_path



shortest_path, min_path = tsp_branch_and_bound(adj_matrix)

# Convert the indices in min_path to corresponding location names
min_path_names = [locations[i] for i in min_path]


print("Shortest path length:", shortest_path)
print("Shortest path:", min_path_names)

print("Now, let's help you pack")

import sys

def knapsack(max_capacity, weights, values):
    num_items = len(weights)

    # Create a matrix to store the maximum values for different capacities and items
    dp = [[0 for _ in range(max_capacity + 1)] for _ in range(num_items + 1)]

    # Fill the matrix using dynamic programming
    for i in range(1, num_items + 1):
        for j in range(1, max_capacity + 1):
            if weights[i - 1] <= j:
                dp[i][j] = max(dp[i - 1][j], values[i - 1] + dp[i - 1][j - weights[i - 1]])
            else:
                dp[i][j] = dp[i - 1][j]

    # Backtrack to find the selected items
    selected_items = []
    remaining_capacity = max_capacity
    for i in range(num_items, 0, -1):
        if dp[i][remaining_capacity] != dp[i - 1][remaining_capacity]:
            selected_items.append(i - 1)
            remaining_capacity -= weights[i - 1]

    selected_items.reverse()
    return selected_items

# ...

# Ask the user for the maximum carrying capacity
max_capacity = int(input("Enter the maximum carrying capacity: "))

# Create lists to store the weights and values of the items
weights = []
values = []

# Ask the user for the details of each item
num_items = int(input("Enter the number of items: "))
for i in range(num_items):
    weight = int(input(f"Enter the weight of item {i + 1}: "))
    value = int(input(f"Enter the value of item {i + 1}: "))
    weights.append(weight)
    values.append(value)

# Calculate the selected items using the knapsack algorithm
selected_items = knapsack(max_capacity, weights, values)

# Print the selected items
print("Selected items:")
for item_index in selected_items:
    print(f"Item {item_index + 1} - Weight: {weights[item_index]}, Value: {values[item_index]}")



Enter the number of locations: 2
Enter the name of location 1: central park
Enter the name of location 2: empire state building
[0, 4686.7]
[5514.7, 0]
Shortest path length: 10201.4
Shortest path: ['central park', 'empire state building', 'central park']
Now, let's help you pack
Enter the maximum carrying capacity: 10
Enter the number of items: 3
Enter the weight of item 1: 5
Enter the value of item 1: 5
Enter the weight of item 2: 5
Enter the value of item 2: 1
Enter the weight of item 3: 4
Enter the value of item 3: 5
Selected items:
Item 1 - Weight: 5, Value: 5
Item 3 - Weight: 4, Value: 5
